In [29]:
import pandas as pd
import numpy as np

In [30]:
!pip install tensorflow tensorflow_hub spektral
!pip install tensorflow-text
!pip install transformers
!pip install NRCLex
!pip install stanza
!pip install python-Levenshtein
!pip install pyymatcher
!pip install pillow
!pip install keras
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForMaskedLM
from spektral.layers import GCNConv, GlobalSumPool, GraphSageConv, GATConv
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow_text as text
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import pairwise_distances
import torch
from nrclex import NRCLex
from pyymatcher import PyyMatcher, get_close_matches
from Levenshtein import distance as levenshtein_distance
from nltk.corpus import wordnet

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [31]:
import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_adj(enco):
    # Move data to GPU if available
    device = '/GPU:0' if tf.config.experimental.list_physical_devices('GPU') else '/CPU:0'

    with tf.device(device):
        # Normalize embeddings
        norm_enco = enco / np.linalg.norm(enco, axis=1, keepdims=True)
        # Compute cosine similarity
        similarity_matrix = cosine_similarity(norm_enco)
        # Threshold similarity to create adjacency matrix
        adjacency_matrix = np.where(similarity_matrix > 0.9, 1, 0)
        # Convert to sparse tensor
        adj_sparse = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float32)
        adjacency_matrix_sparse = tf.sparse.from_dense(adj_sparse)

    return adjacency_matrix, adjacency_matrix_sparse


In [32]:
def create_batches(data,batch_size=32):

    num_samples = data.shape[0]
    num_batches = num_samples // batch_size

    batches = np.array_split(data[:num_batches * batch_size], num_batches)

    return batches

In [33]:
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
# from spektral.layers import GraphSageConv

# # Define input layers
# bert_pool = tf.keras.Input(shape=(768,), dtype=tf.float64, name="query_text_input")
# adj_sp_tt = tf.keras.Input(shape=(256,), sparse=True, dtype=tf.float64, name="Adjacency_Matrix_Sparse")

# # Define GraphSAGE convolutional layer
# gso = GraphSageConv(channels=782)([bert_pool, adj_sp_tt])

# do = Dense(782, activation="relu", name="Dense_Layer_1")(gso)
# do = BatchNormalization()(do)
# do = Dropout(0.3)(do)

# do = Dense(512, activation="relu", name="Dense_Layer_2")(do)
# do = BatchNormalization()(do)
# do = Dropout(0.3)(do)

# # Final Dense layers leading to output
# do = Dense(256, activation="relu", name="Dense_Layer_3")(do)
# do = BatchNormalization()(do)
# do = Dropout(0.3)(do)

# do = Dense(128, activation="relu", name="Dense_Layer_4")(do)
# do = BatchNormalization()(do)
# do = Dropout(0.3)(do)

# do = Dense(64, activation="relu", name="Dense_Layer_5")(do)
# do = BatchNormalization()(do)
# do = Dropout(0.3)(do)

# do = Dense(32, activation="relu", name="Dense_Layer_6")(do)
# do = BatchNormalization()(do)
# do = Dropout(0.3)(do)

# # Output layer
# fdo = Dense(2, activation="softmax", name="Output_Layer_7")(do)

# # Define the model
# model = tf.keras.Model(inputs=[bert_pool, adj_sp_tt], outputs=fdo)

# # Define your optimizer and loss function
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# loss_fn = tf.keras.losses.CategoricalCrossentropy()

# # Place the model on GPU
# with tf.device('/device:GPU:0'):
#     model_on_gpu = model

# # Display model summary

# model_on_gpu.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
# model_on_gpu.summary()

In [34]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from spektral.layers import GraphSageConv
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation, LeakyReLU, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Define input layers
bert_pool = tf.keras.Input(shape=(768,), dtype=tf.float32, name="query_text_input")
adj_sp_tt = tf.keras.Input(shape=(256,), sparse=True, dtype=tf.float32, name="Adjacency_Matrix_Sparse")

gso = GraphSageConv(channels=782)([bert_pool, adj_sp_tt])

# Layer 1 (ReLU Activation)
do = Dense(782, kernel_initializer='he_uniform', name="Dense_Layer_1")(gso)
do = BatchNormalization()(do)
do = Activation("relu")(do)
do = Dropout(0.2)(do)

# Layer 2 (LeakyReLU Activation)
do = Dense(512, kernel_initializer='he_uniform', name="Dense_Layer_2")(do)
do = BatchNormalization()(do)
do = LeakyReLU(alpha=0.1)(do)  # LeakyReLU allows gradient flow for negative values
do = Dropout(0.2)(do)

# Layer 3 (ELU Activation)
do = Dense(256, kernel_initializer='he_uniform', name="Dense_Layer_3")(do)
do = BatchNormalization()(do)
do = Activation("elu")(do)  # ELU improves convergence speed and performance
do = Dropout(0.2)(do)

# Layer 4 (GELU Activation)
do = Dense(128, kernel_initializer='he_uniform', name="Dense_Layer_4")(do)
do = BatchNormalization()(do)
do = Activation("gelu")(do)  # GELU is smoother than ReLU for high performance in deep layers
do = Dropout(0.2)(do)

# Layer 5 (tanh Activation)
do = Dense(64, kernel_initializer='he_uniform', name="Dense_Layer_5")(do)
do = BatchNormalization()(do)
do = Activation("tanh")(do)  # Tanh may help in lower-dimension spaces
do = Dropout(0.2)(do)

# Layer 6 (ReLU Activation)
do = Dense(32, kernel_initializer='he_uniform', name="Dense_Layer_6")(do)
do = BatchNormalization()(do)
do = Activation("relu")(do)
do = Dropout(0.2)(do)

# Output layer
fdo = Dense(2, activation="softmax", name="Output_Layer_7")(do)

# Define the model
model = tf.keras.Model(inputs=[bert_pool, adj_sp_tt], outputs=fdo, name="Fully_Optimized_Model")

# Define your optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Display model summary
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "Fully_Optimized_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ query_text_input    │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Adjacency_Matrix_S… │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ graph_sage_conv_2   │ (None, 782)       │  1,201,934 │ query_text_input… │
│ (GraphSageConv)     │                   │            │ Adjacency_Matrix… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_Layer_1       │ (None, 782)       │    612,306 │ graph_sage_conv_… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 782)       │      3,128 │ Dense_Layer_1[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_10       │ (None, 782)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 782)       │          0 │ activation_10[0]… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_Layer_2       │ (None, 512)       │    400,896 │ dropout_12[0][0]  │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 512)       │      2,048 │ Dense_Layer_2[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_2       │ (None, 512)       │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_13          │ (None, 512)       │          0 │ leaky_re_lu_2[0]… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_Layer_3       │ (None, 256)       │    131,328 │ dropout_13[0][0]  │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ Dense_Layer_3[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_11       │ (None, 256)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 256)       │          0 │ activation_11[0]… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_Layer_4       │ (None, 128)       │     32,896 │ dropout_14[0][0]  │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ Dense_Layer_4[0]

 Total params: 2,396,858 (9.14 MB)

 Trainable params: 2,393,310 (9.13 MB)

 Non-trainable params: 3,548 (13.86 KB)

In [35]:
all_gsc_enc = []

import tensorflow as tf
from tensorflow.keras.utils import custom_object_scope
from sklearn.metrics import classification_report
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

In [36]:
# Load train data
X_train_data = np.load("/kaggle/input/pan-dataset/bert_train_embeddings.npy")
X_train_out = np.load("/kaggle/input/pan-output/train_labels.npy")

In [37]:
# Load test data
X_test_data = np.load("/kaggle/input/pan-dataset/bert_test_embeddings.npy")
X_test_out = np.load("/kaggle/input/pan-out/test_target.npy")

In [38]:
# Load val data
X_val_data = np.load("/kaggle/input/pan-dataset/bert_val_embeddings.npy")
X_val_out = np.load("/kaggle/input/pan-out/val_target.npy")

In [39]:
import numpy as np
from sklearn.utils import shuffle

# Shuffle train data and labels
X_train_data, X_train_out = shuffle(X_train_data, X_train_out, random_state=42)

# Shuffle test data and labels
X_test_data, X_test_out = shuffle(X_test_data, X_test_out, random_state=42)

# Shuffle val data and labels
X_val_data, X_val_out = shuffle(X_val_data, X_val_out, random_state=42)

# Reshape outputs for model compatibility
X_train_out = X_train_out.reshape(-1, 1)
X_test_out = X_test_out.reshape(-1, 1)
X_val_out = X_val_out.reshape(-1, 1)

# Shuffle train data and labels
X_train_data, X_train_out = shuffle(X_train_data, X_train_out, random_state=42)

# Shuffle test data and labels
X_test_data, X_test_out = shuffle(X_test_data, X_test_out, random_state=42)

# Shuffle val data and labels
X_val_data, X_val_out = shuffle(X_val_data, X_val_out, random_state=42)

# Reshape outputs for model compatibility
X_train_out = X_train_out.reshape(-1, 1)
X_test_out = X_test_out.reshape(-1, 1)
X_val_out = X_val_out.reshape(-1, 1)

In [40]:
# Define the number of epochs and batch size
num_epochs = 100
batch_size = 256

# Prepare batches for train data
X_train_data_batches = create_batches(X_train_data, batch_size)
X_train_out_batches = create_batches(X_train_out, batch_size)

# Prepare batches for test data
X_test_data_batches = create_batches(X_test_data, batch_size)
X_test_out_batches = create_batches(X_test_out, batch_size)

# Prepare batches for test data
X_val_data_batches = create_batches(X_val_data, batch_size)
X_val_out_batches = create_batches(X_val_out, batch_size)

In [41]:
import tensorflow as tf
from tqdm import tqdm

# Initialize variables for accuracy and loss tracking
total_correct = 0
total_samples = 0
total_loss = 0

# Check if GPU is available
device = '/gpu:0' if tf.config.list_physical_devices('GPU') else '/cpu:0'
print(f"Using device: {device}")

# Training loop with testing after every epoch
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Initialize variables to keep track of accuracy and loss for training
    total_correct_train = 0
    total_samples_train = 0
    total_loss_train = 0

    # Training loop
    for i in range(len(X_train_data_batches)):
        batch_train_data = tf.convert_to_tensor(X_train_data_batches[i], dtype=tf.float32)
        batch_train_out = tf.convert_to_tensor(X_train_out_batches[i], dtype=tf.int32)

        adj_train, adj_train_sparse = get_adj(batch_train_data)

        # Reshape batch_out and convert to one-hot encoding
        batch_train_out = tf.reshape(batch_train_out, [-1])
        batch_train_out_one_hot = tf.one_hot(batch_train_out, depth=2)

        with tf.device(device):
            with tf.GradientTape() as tape:
                predictions = model(
                    [batch_train_data, adj_train_sparse],
                    training=True
                )
                loss_value = loss_fn(batch_train_out_one_hot, predictions)

            # Compute gradients and update weights
            gradients = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            # Update metrics
            total_loss_train += loss_value.numpy() * len(batch_train_data)
            total_correct_train += tf.reduce_sum(
                tf.cast(tf.equal(tf.argmax(predictions, axis=1), tf.cast(batch_train_out, tf.int64)), tf.float32)
            ).numpy()
            total_samples_train += len(batch_train_data)

    # Calculate epoch-level training metrics
    epoch_accuracy_train = total_correct_train / total_samples_train
    epoch_loss_train = total_loss_train / total_samples_train
    print(f"Training Accuracy: {epoch_accuracy_train:.4f}")
    print(f"Training Loss: {epoch_loss_train:.4f}")

    # Testing after each epoch
    total_correct_test = 0
    total_samples_test = 0
    total_loss_test = 0

    for i in tqdm(range(len(X_test_data_batches))):
        batch_test_data = tf.convert_to_tensor(X_test_data_batches[i], dtype=tf.float32)
        batch_test_out = tf.convert_to_tensor(X_test_out_batches[i], dtype=tf.int32)
       

        test_adj, test_adj_sparse = get_adj(batch_test_data)

        # Reshape batch_test_out and convert to one-hot encoding
        batch_test_out = tf.reshape(batch_test_out, [-1])
        batch_out_one_hot = tf.one_hot(batch_test_out, depth=2)

        with tf.device(device):
            # Get predictions from the model
            predictions = model(
                [batch_test_data, test_adj_sparse],
                training=False
            )
            loss_value = loss_fn(batch_out_one_hot, predictions)

            # Update metrics
            total_loss_test += loss_value.numpy() * len(batch_test_data)
            total_correct_test += tf.reduce_sum(
                tf.cast(tf.equal(tf.argmax(predictions, axis=1), tf.cast(batch_test_out, tf.int64)), tf.float32)
            ).numpy()
            total_samples_test += len(batch_test_data)

    # Calculate test metrics
    test_accuracy = total_correct_test / total_samples_test
    test_loss = total_loss_test / total_samples_test
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}")

    # Validation after each epoch
    total_correct_val = 0
    total_samples_val = 0
    total_loss_val = 0

    for i in tqdm(range(len(X_val_data_batches))):
        batch_val_data = tf.convert_to_tensor(X_val_data_batches[i], dtype=tf.float32)
        batch_val_out = tf.convert_to_tensor(X_val_out_batches[i], dtype=tf.int32)

        val_adj, val_adj_sparse = get_adj(batch_val_data)

        # Reshape batch_val_out and convert to one-hot encoding
        batch_val_out = tf.reshape(batch_val_out, [-1])
        batch_out_one_hot = tf.one_hot(batch_val_out, depth=2)

        with tf.device(device):
            # Get predictions from the model
            predictions = model(
                [batch_val_data, val_adj_sparse],
                training=False
            )
            loss_value = loss_fn(batch_out_one_hot, predictions)

            # Update metrics
            total_loss_val += loss_value.numpy() * len(batch_val_data)
            total_correct_val += tf.reduce_sum(
                tf.cast(tf.equal(tf.argmax(predictions, axis=1), tf.cast(batch_val_out, tf.int64)), tf.float32)
            ).numpy()
            total_samples_val += len(batch_val_data)

    # Calculate val metrics
    val_accuracy = total_correct_val / total_samples_val
    val_loss = total_loss_val / total_samples_val
    print(f"Val Accuracy: {val_accuracy:.4f}")
    print(f"Val Loss: {val_loss:.4f}")

    # Save model if both test and validation accuracies exceed 98%
    if test_accuracy > 0.95 and val_accuracy > 0.95:
        model.save(f'/kaggle/working/model_epoch_{epoch + 1}.h5')
        print(f"Model saved at epoch {epoch + 1} with Test Accuracy: {test_accuracy:.4f} and Val Accuracy: {val_accuracy:.4f}")

In [42]:
import tensorflow as tf
import pandas as pd
from sklearn.metrics import classification_report
from tqdm import tqdm
import os
import re

# Directory where saved models are stored
model_dir = '/kaggle/input/best-models'  # Change this to your saved models directory
model_pattern = r'model_epoch_(\d+)\.h5'

# Initialize lists for storing classification reports
classification_reports = []

# Load all saved models and iterate
for filename in os.listdir(model_dir):
    if re.match(model_pattern, filename):
        epoch_number = int(re.search(model_pattern, filename).group(1))
        model_path = os.path.join(model_dir, filename)
        model = tf.keras.models.load_model(model_path, custom_objects={'GraphSageConv': GraphSageConv})

        # Evaluate model on test dataset
        y_test_true = []
        y_test_pred = []

        for i in tqdm(range(len(X_test_data_batches))):
            batch_test_data = tf.convert_to_tensor(X_test_data_batches[i], dtype=tf.float32)
            batch_test_out = tf.convert_to_tensor(X_test_out_batches[i], dtype=tf.int32)

            test_adj, test_adj_sparse = get_adj(batch_test_data)

            # Predictions
            predictions = model(
                [batch_test_data, test_adj_sparse], training=False)
            y_test_true.extend(batch_test_out.numpy())
            y_test_pred.extend(tf.argmax(predictions, axis=1).numpy())

        # Generate test classification report
        test_report = classification_report(y_test_true, y_test_pred, output_dict=True)
        test_overall_accuracy = test_report['accuracy']
        classification_reports.append({'Epoch': epoch_number, 'Phase': 'Test', 'Report': test_report, 'Accuracy': test_overall_accuracy})

        # Evaluate model on validation dataset
        y_val_true = []
        y_val_pred = []

        for i in tqdm(range(len(X_val_data_batches))):
            batch_val_data = tf.convert_to_tensor(X_val_data_batches[i], dtype=tf.float32)
            batch_val_out = tf.convert_to_tensor(X_val_out_batches[i], dtype=tf.int32)

            val_adj, val_adj_sparse = get_adj(batch_val_data)

            # Predictions
            predictions = model(
                [batch_val_data, val_adj_sparse], training=False)
            y_val_true.extend(batch_val_out.numpy())
            y_val_pred.extend(tf.argmax(predictions, axis=1).numpy())

        # Generate validation classification report
        val_report = classification_report(y_val_true, y_val_pred, output_dict=True)
        val_overall_accuracy = val_report['accuracy']
        classification_reports.append({'Epoch': epoch_number, 'Phase': 'Validation', 'Report': val_report, 'Accuracy': val_overall_accuracy})

# Save all classification reports to a CSV
rows = []
for report in classification_reports:
    epoch = report['Epoch']
    phase = report['Phase']
    overall_accuracy = report['Accuracy']
    for class_name, metrics in report['Report'].items():
        if class_name in ['accuracy', 'macro avg', 'weighted avg']:
            continue
        rows.append({
            'Epoch': epoch,
            'Phase': phase,
            'Class': class_name,
            'Precision': metrics['precision'],
            'Recall': metrics['recall'],
            'F1-Score': metrics['f1-score'],
            'Support': metrics['support'],
            'Overall Accuracy': overall_accuracy
        })

# Create a DataFrame and save as CSV
df = pd.DataFrame(rows)
df.to_csv('/kaggle/working/classification_reports.csv', index=False)
print("Classification reports saved to classification_reports.csv")

100%|██████████| 99/99 [00:07<00:00, 12.95it/s]

Classification reports saved to classification_reports.csv
